# PySimPace: Machine Learning Integration Tutorial

This notebook demonstrates how to use PySimPace to:

✅ Generate paired (clean, motion-corrupted) MRI data  
✅ Prepare datasets for deep learning pipelines  
✅ Load data with a standard PyTorch Dataset  
✅ Visualize the output  

---

PySimPace supports both **structural MRI (3D)** and **fMRI (4D)** motion simulation.  
In this tutorial, we will generate a dataset for structural MRI.

---

👉 The generated data is ideal for training models such as:
- Motion correction U-Nets
- Artifact detection classifiers
- Segmentation models robust to motion

---


In [2]:
# If using locally:
# pip install -e .

# If PySimPace is already installed:
# pip install pysimpace

In [8]:
! pip install py-simpace

In [14]:
import os
from pysimpace.ml import generate_training_pairs, MRIPairedDataset

# Paths (adjust these as needed)
clean_dir = os.path.join("clean")
output_dir = os.path.join("errored")

print("Clean data folder:", clean_dir)
print("Output folder:", output_dir)

Clean data folder: clean
Output folder: errored


In [15]:
generate_training_pairs(
    clean_dir=clean_dir,
    output_dir=output_dir,
    n_samples=5,  # start small for test
    artifact_configs=None,   # use default
    structural=True,
    use_blended=True,
    save_format='nifti',
    seed=42
)

Generating pairs:   0%|                                                                                       | 0/5 [00:00<?, ?it/s]


TypeError: simulate_structural_motion() got an unexpected keyword argument 'apply_spike_noise'